# Building null model with GMMAT

Author: Jose Jaime Martinez-Magana

Day: 02 April 2023

This script was developed to build the null model with GMMAT.

In [ ]:
# move to path
cd /vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/pca_grm

# request computational resources
srun --pty --mem=32G -p interactive bash

# load miniconda
module load miniconda

# load environment
conda activate tractor_mix

# start R
R

# load GMMAT
library(GMMAT)

# read phenofile
pheno=read.csv("/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/phenofile/tractor_mix_pheno_alc.csv")

# read pca and GRM
grm=readRDS("GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.gds_prunned_grm_pca_v03292023.rds")

# fitting null model for heigth
# merging PCs for model
pcs=grm$PCair$vectors[,c(1:10)]
pcs=as.data.frame(pcs)
colnames(pcs)=paste0(rep("PC",10),rep(1:10)) 
# add sample ID
pcs$SampleID=rownames(pcs)

# remove FID from pheno file
pheno$SampleID=gsub(".*_","",pheno$FID_IID)

# merging 
pheno_pcs=merge(pheno,pcs, by=c("SampleID"))

# verify the order of the matrix
pheno_pcs$SampleID

# extract ordering index
reorde_idx=match(pheno_pcs$SampleID, rownames(grm$grm_sparse))

# reorder pheno based on index
pheno_pcs_or=pheno_pcs[reorde_idx,]

# fitting null model with ordered phenofile
model_null=glmmkin(fixed = alcdep2 ~ age + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10,
                  data=pheno_pcs_or, id="SampleID",kins=grm$grm_sparse,
                  family=binomial(link = "logit"))


# save linear null model 1
saveRDS(file="GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.gds_prunned_alc.null.model.rds", model_null)

# modify null model to adjust SampleID for FID_IID
model_null_fid=model_null

# adjust sampleID to FID_IID
model_null_fid$id_include=pheno_pcs$FID_IID

# save linear null model 1
saveRDS(file="GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.gds_prunned_alc_fid.null.model.rds", model_null_fid)

# exit R
q()